In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#lettura del file

file_path_movies = "Elenco_Movies_SAC.csv"
sdf = pd.read_csv(file_path_movies)
sdf

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [3]:
file_path_users = "users_edit.csv"
sdf_users = pd.read_csv(file_path_users)
sdf_users

,UserID,Gender,Age,CAP,Work
0,1,F,11,20023,Studente
1,2,M,13,72022,Studente
2,3,M,24,36100,Autista
3,4,M,37,85024,Data Engineer
4,5,M,15,1010,Studente
...,...,...,...,...,...
6035,6036,F,33,12032,Autista
6036,6037,F,59,9099,Dirigente
6037,6038,F,35,19020,Dirigente
6038,6039,F,52,1010,Disoccupato


In [4]:
file_path_ratings = "ratings_edit.csv"
sdf_ratings = pd.read_csv(file_path_ratings)
sdf_ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
sdf_ratings["Timestamp"] = pd.to_datetime(sdf_ratings["Timestamp"], unit='s')
sdf_ratings["Timestamp"] = sdf_ratings["Timestamp"].dt.date
sdf_ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,2000-12-31
1,1,661,3,2000-12-31
2,1,914,3,2000-12-31
3,1,3408,4,2000-12-31
4,1,2355,5,2001-01-06
...,...,...,...,...
1000204,6040,1091,1,2000-04-26
1000205,6040,1094,5,2000-04-25
1000206,6040,562,5,2000-04-25
1000207,6040,1096,4,2000-04-26


In [6]:
#Errore 1: Ribadiamo la richiesta di creare una nuova colonna YEAR estrapolandolo dalla colonna Title

sdf["Year"] = sdf["Title"].str.extract(r'\((\d{4})\)', expand = False)
sdf["Title"] = sdf["Title"].str.replace(r'\((\d{4})\)', '', regex = True).str.strip()
sdf

,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
3878,3948,Meet the Parents,Comedy,2000
3879,3949,Requiem for a Dream,Drama,2000
3880,3950,Tigerland,Drama,2000
3881,3951,Two Family House,Drama,2000


In [7]:
#Errore 2: Il film con anno “non_def” , è dell’anno 1995
sdf[sdf["Year"].isna()]
sdf.loc[15, "Year"] = "1995"
sdf.loc[15, "Title"] = sdf.loc[15, "Title"].replace("(not_def)", "").strip()
sdf.loc[15]
sdf["Year"] = pd.to_numeric(sdf["Year"])
sdf["Year"] = sdf["Year"].astype(int)

In [8]:
#Errore 3: Il film senza titolo si intitola “Phantom of the Opera, The”

sdf[sdf["Title"]==""]
sdf.loc[sdf["MovieID"] == 3936, "Title"] = "Phantom of the Opera, The"
sdf[sdf["MovieID"]== 3936]

,MovieID,Title,Genres,Year
3866,3936,"Phantom of the Opera, The",Drama|Thriller,1943


In [9]:
#Errore 4: C’è un film che ha i generi separati da “–” e non da “|” come gli altri, quindi è necessario sistemare l’errore.
sdf['Genres'] = sdf['Genres'].str.replace('--', '|', regex=False)

In [10]:
#Errore 5: Ci sono 5 film di genere “Drama” con un errore ortografico nella parola “Drama” della colonna “Genre”, chiediamo di allinearli agli altri rinominando il genere “Drama”.

sdf.loc[[604, 3248], "Genres"] = "Drama"

In [11]:
#Correzione dei titoli con apici
sdf['Title'] = sdf['Title'].str.replace(r'^"(.+)"$', "", regex=True)
sdf[sdf["Genres"].str.contains("Dramatic")]
sdf.loc[[523, 3340, 3509], "Genres"] = "Drama"

In [12]:
#Creazione degli input per il db movies

table_name = "movie_table"
sql_statements_movies = [
    f"INSERT INTO {table_name} ({', '.join(sdf.columns)}) VALUES ({', '.join(map(repr, row))});"
    for row in sdf.itertuples(index=False, name=None)
]

with open("output_movies.sql", "w") as f:
    f.write("\n".join(sql_statements_movies))

print("\n".join(sql_statements_movies)) 

INSERT INTO movie_table (MovieID, Title, Genres, Year) VALUES (1, 'Toy Story', "Animation|Children's|Comedy", 1995);
INSERT INTO movie_table (MovieID, Title, Genres, Year) VALUES (2, 'Jumanji', "Adventure|Children's|Fantasy", 1995);
INSERT INTO movie_table (MovieID, Title, Genres, Year) VALUES (3, 'Grumpier Old Men', 'Comedy|Romance', 1995);
INSERT INTO movie_table (MovieID, Title, Genres, Year) VALUES (4, 'Waiting to Exhale', 'Comedy|Drama', 1995);
INSERT INTO movie_table (MovieID, Title, Genres, Year) VALUES (5, 'Father of the Bride Part II', 'Comedy', 1995);
INSERT INTO movie_table (MovieID, Title, Genres, Year) VALUES (6, 'Heat', 'Action|Crime|Thriller', 1995);
INSERT INTO movie_table (MovieID, Title, Genres, Year) VALUES (7, 'Sabrina', 'Comedy|Romance', 1995);
INSERT INTO movie_table (MovieID, Title, Genres, Year) VALUES (8, 'Tom and Huck', "Adventure|Children's", 1995);
INSERT INTO movie_table (MovieID, Title, Genres, Year) VALUES (9, 'Sudden Death', 'Action', 1995);
INSERT INTO 

In [13]:
#Creazione degli input per il db users

table_name = "user_table"
sql_statements_users = [
    f"INSERT INTO {table_name} ({', '.join(sdf_users.columns)}) VALUES ({', '.join(map(repr, row))});"
    for row in sdf_users.itertuples(index=False, name=None)
]

with open("output_users.sql", "w") as f:
    f.write("\n".join(sql_statements_users))

print("\n".join(sql_statements_users)) 

INSERT INTO user_table (UserID, Gender, Age, CAP, Work) VALUES (1, 'F', 11, 20023, 'Studente');
INSERT INTO user_table (UserID, Gender, Age, CAP, Work) VALUES (2, 'M', 13, 72022, 'Studente');
INSERT INTO user_table (UserID, Gender, Age, CAP, Work) VALUES (3, 'M', 24, 36100, 'Autista');
INSERT INTO user_table (UserID, Gender, Age, CAP, Work) VALUES (4, 'M', 37, 85024, 'Data Engineer');
INSERT INTO user_table (UserID, Gender, Age, CAP, Work) VALUES (5, 'M', 15, 1010, 'Studente');
INSERT INTO user_table (UserID, Gender, Age, CAP, Work) VALUES (6, 'F', 36, 153, 'Data Scientis');
INSERT INTO user_table (UserID, Gender, Age, CAP, Work) VALUES (7, 'M', 24, 15067, 'Dirigente');
INSERT INTO user_table (UserID, Gender, Age, CAP, Work) VALUES (8, 'M', 28, 44020, 'Avvocato');
INSERT INTO user_table (UserID, Gender, Age, CAP, Work) VALUES (9, 'M', 11, 10126, 'Studente');
INSERT INTO user_table (UserID, Gender, Age, CAP, Work) VALUES (10, 'F', 65, 9010, 'Dirigente');
INSERT INTO user_table (UserID, 

In [14]:
#Creazione degli input per il db ratings

#table_name = "ratings_table"
#sql_statements_ratings = [
#    f"INSERT INTO {table_name} ({', '.join(sdf_ratings.columns)}) VALUES ({', '.join(map(repr, row))});"
#    for row in sdf_ratings.itertuples(index=False, name=None)
#]
#
#with open("output_ratings.sql", "w") as f:
#    f.write("\n".join(sql_statements_ratings))
#
#print("\n".join(sql_statements_ratings))


In [15]:
#Installazione di mysql-connector in jupiter

#!pip install mysql-connector-python

In [16]:
#Importazione mysql_connector
import mysql.connector

In [17]:
# Correct way to define configuration
config = {
    'host': "localhost",
    'user': "root",
    'password': "KopiNf23@",
    'database': "on_the_movie"
}

# Establishing connection
with mysql.connector.connect(**config) as conn:
    with conn.cursor() as cursor:
        # Fetch all records from movie_table
        query = "SELECT * FROM movie_table"
        cursor.execute(query)
        records = cursor.fetchall()
        print("Movie Table Records:", records)
#
        query = "SELECT * FROM genres_table"
        cursor.execute(query)
        records = cursor.fetchall()
        print("Genres Table Records:", records)

Movie Table Records: []
Genres Table Records: [(1, 'Animation'), (2, "Children's"), (3, 'Comedy'), (4, 'Adventure'), (5, 'Fantasy'), (6, 'Romance'), (7, 'Drama'), (8, 'Action'), (9, 'Crime'), (10, 'Thriller'), (11, 'Horror'), (12, 'Sci-Fi'), (13, 'Documentary'), (14, 'War'), (15, 'Musical'), (16, 'Mystery'), (17, 'Film-Noir'), (18, 'Western')]


In [18]:
# insert into movie_table
import mysql.connector
with mysql.connector.connect(**config) as conn:
    with conn.cursor() as cursor:
        
        query_movie= "insert into movie_table(MovieID, Title, Genres, Year) values (%s,%s,%s,%s)"
   
        for tupla in sdf.itertuples(index=False, name=None):
            
            cursor.execute(query_movie, (tupla[0],tupla[1],tupla[2],tupla[3]))
    conn.commit()


In [19]:
# insert into users_table
import mysql.connector
with mysql.connector.connect(**config) as conn:
    with conn.cursor() as cursor:
        
        query_users= "insert into users_table(UsersId, Gender, Age, CAP, Work) values (%s,%s,%s,%s,%s)"
   
        for tupla in sdf_users.itertuples(index=False, name=None):
            
            cursor.execute(query_users, (tupla[0],tupla[1],tupla[2],tupla[3],tupla[4]))
    conn.commit()


In [21]:
# insert into users_ratings
with mysql.connector.connect(**config) as conn:
    with conn.cursor() as cursor:
        
        query_ratings= "insert into ratings_table(UserID, MovieID, Rating, Timestamp) values (%s,%s,%s,%s)"
   
        for tupla in sdf_ratings.itertuples(index=False, name=None):
            
            cursor.execute(query_ratings, (tupla[0],tupla[1],tupla[2],tupla[3]))
    conn.commit()

In [22]:
sdf["Genres"].str.split("|").explode().unique()

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [23]:
dizionario = dict(enumerate(sdf["Genres"].str.split("|").explode().unique(), start = 1))
dizionario

{1: 'Animation',
 2: "Children's",
 3: 'Comedy',
 4: 'Adventure',
 5: 'Fantasy',
 6: 'Romance',
 7: 'Drama',
 8: 'Action',
 9: 'Crime',
 10: 'Thriller',
 11: 'Horror',
 12: 'Sci-Fi',
 13: 'Documentary',
 14: 'War',
 15: 'Musical',
 16: 'Mystery',
 17: 'Film-Noir',
 18: 'Western'}

In [24]:
# dizionario = {i + 1: val for i, val in enumerate(sdf["Genres"].str.split("|").explode().unique())}
# dizionario

In [25]:

with mysql.connector.connect(**config) as conn:
    with conn.cursor() as cursor:
        query_movies_genres = "insert into movies_genres (genres_id, movie_id) values (%s,%s)"
        for x in sdf.itertuples(index=False, name=None):
            # print(x)
            for g in dizionario.items():
                if g[1] in x[2]:
                    cursor.execute(query_movies_genres, (g[0], x[0]))
    conn.commit()

In [26]:
# insert into age_ranges
with mysql.connector.connect(**config) as conn:
    with conn.cursor() as cursor:
        cursor.execute("select AgeId, Age_ranges from age_range_table")
        age_ranges = cursor.fetchall()
        age_map = {}
        for age_id, age_range in age_ranges:
            if "<" in age_range:
                min_age = 0
                max_age = int(age_range.replace("<", ""))
            elif ">" in age_range:
                min_age = int(age_range.replace(">", ""))
                max_age = 200
            else:
                min_age, max_age = map(int, age_range.split("-"))

            age_map[age_id] = (min_age, max_age)


        for age_id, (min_age, max_age) in age_map.items():
            update_query = """
            UPDATE users_table 
            SET age_range = %s 
            WHERE Age BETWEEN %s AND %s
            """
            cursor.execute(update_query, (age_id, min_age, max_age))
    

        conn.commit()

In [27]:
import mysql.connector
with mysql.connector.connect(**config) as conn:
    with conn.cursor() as cursor:
        cursor.execute("select * from age_range_table")
        results = cursor.fetchall()
        for a in sdf_users["Age"]:
            query_update = "update users_table set age_range = %s where Age = %s"
            if a < 18:
                cursor.execute(query_update, (results[0][0],a))
            elif a >=18 and a <= 24:
                cursor.execute(query_update, (results[1][0],a))
            elif a >= 25 and a<=34:
                cursor.execute(query_update, (results[2][0],a))
            elif a >= 35 and a<=44:
                cursor.execute(query_update, (results[3][0],a))
            elif a >= 45 and a<=54:
                cursor.execute(query_update, (results[4][0],a))
            else: 
                cursor.execute(query_update, (results[5][0],a))
            
    conn.commit()